In [2]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
plt.style.use("seaborn-v0_8-whitegrid")

In [3]:
path = "../data/raw/imports-exports-commerciaux.csv"

encodings = ["utf-8", "latin-1"]
seps = [",", ";", "\t", "|"]
df, last_err = None, None

for enc in encodings:
    for sep in seps:
        try:
            tmp = pd.read_csv(path, sep=sep, encoding=enc)
            if tmp.shape[1] >= 6:
                df = tmp
                print(f"✅ Lecture réussie - encoding='{enc}'")
                break
        except Exception as e:
            last_err = e
    if df is not None:
        break

if df is None:
    raise RuntimeError(f"Impossible de lire le CSV: {last_err}")

df.head()

✅ Lecture réussie - encoding='utf-8'


,Date,Tranche horaire du programme d'échange,FR vers GB (MWh),GB vers FR (MWh),FR vers CH (MWh),CH vers FR (MWh),FR vers IT (MWh),IT vers FR (MWh),FR vers ES (MWh),ES vers FR (MWh),Export France (MWh),Import France (MWh),CWE/Core->FR,FR->CWE/Core
0,2019-05-11,4.0,1500.0,-99.0,2529.0,-21.4,2241.0,0.0,1800.0,-600.0,NaN,NaN,-635.6,5912.9
1,2024-05-11,14.0,3028.0,-1344.0,1270.3,-1120.3,159.0,0.0,2914.0,-278.0,NaN,NaN,-6533.5,1596.4
2,2005-10-02,20.0,1999.0,0.0,2996.0,-279.0,2235.0,0.0,303.0,-577.0,NaN,NaN,-1944.0,2903.0
3,2007-10-02,16.0,1484.0,0.0,3127.0,-381.0,2648.0,0.0,305.0,-608.0,NaN,NaN,-2122.0,1869.0
4,2011-10-02,12.0,500.0,0.0,2947.0,-138.0,2465.0,0.0,1219.0,-129.0,NaN,NaN,-1287.0,2447.0


In [4]:
# Standardiser les noms de colonnes
df.columns = [c.strip() for c in df.columns]

# Parsing de la date
df["Date"] = pd.to_datetime(df["Date"], errors="coerce", dayfirst=True)

# Parsing de la tranche horaire
if "Tranche horaire du programme d'échange" in df.columns:
    df["hour"] = pd.to_numeric(df["Tranche horaire du programme d'échange"], errors="coerce").fillna(0).astype(int)
    df["datetime"] = df["Date"] + pd.to_timedelta(df["hour"].clip(0, 23), unit="h")
else:
    df["datetime"] = df["Date"]

df = df.sort_values("datetime")
df.head()

,Date,Tranche horaire du programme d'échange,FR vers GB (MWh),GB vers FR (MWh),FR vers CH (MWh),CH vers FR (MWh),FR vers IT (MWh),IT vers FR (MWh),FR vers ES (MWh),ES vers FR (MWh),Export France (MWh),Import France (MWh),CWE/Core->FR,FR->CWE/Core,hour,datetime
143694,2005-01-01,1.0,2000.0,0.0,3093.0,-874.0,2432.0,0.0,1405.0,-22.0,NaN,NaN,-1966.0,2288.0,1,2005-01-01 01:00:00
166337,2005-01-01,2.0,2000.0,0.0,3093.0,-814.0,2432.0,0.0,1405.0,-22.0,NaN,NaN,-2045.0,2238.0,2,2005-01-01 02:00:00
9956,2005-01-01,3.0,2000.0,0.0,3093.0,-750.0,2432.0,0.0,1405.0,-22.0,NaN,NaN,-2057.0,2238.0,3,2005-01-01 03:00:00
76821,2005-01-01,4.0,2000.0,0.0,3113.0,-698.0,2432.0,0.0,1405.0,-22.0,NaN,NaN,-2038.0,2010.0,4,2005-01-01 04:00:00
126009,2005-01-01,5.0,2000.0,0.0,3113.0,-693.0,2432.0,0.0,1405.0,-22.0,NaN,NaN,-2026.0,2010.0,5,2005-01-01 05:00:00


In [5]:
df["year"] = df["datetime"].dt.year
df["month"] = df["datetime"].dt.to_period("M").astype(str)
df["day"] = df["datetime"].dt.date
df["hour"] = df["datetime"].dt.hour

df[["datetime", "year", "month", "day", "hour"]].head()

,datetime,year,month,day,hour
143694,2005-01-01 01:00:00,2005.0,2005-01,2005-01-01,1.0
166337,2005-01-01 02:00:00,2005.0,2005-01,2005-01-01,2.0
9956,2005-01-01 03:00:00,2005.0,2005-01,2005-01-01,3.0
76821,2005-01-01 04:00:00,2005.0,2005-01,2005-01-01,4.0
126009,2005-01-01 05:00:00,2005.0,2005-01,2005-01-01,5.0


In [6]:
partners = [
    ("FR vers GB (MWh)", "GB vers FR (MWh)", "GBR"),
    ("FR vers CH (MWh)", "CH vers FR (MWh)", "CHE"),
    ("FR vers IT (MWh)", "IT vers FR (MWh)", "ITA"),
    ("FR vers ES (MWh)", "ES vers FR (MWh)", "ESP"),
]

for e_col, i_col, code in partners:
    if e_col in df.columns and i_col in df.columns:
        df[f"net_{code}"] = df[e_col].fillna(0) - df[i_col].fillna(0)

# Solde total
df["net_total"] = df[[c for c in df.columns if c.startswith("net_")]].sum(axis=1)
df.head()

,Date,Tranche horaire du programme d'échange,FR vers GB (MWh),GB vers FR (MWh),FR vers CH (MWh),CH vers FR (MWh),FR vers IT (MWh),IT vers FR (MWh),FR vers ES (MWh),ES vers FR (MWh),Export France (MWh),Import France (MWh),CWE/Core->FR,FR->CWE/Core,hour,datetime,year,month,day,net_GBR,net_CHE,net_ITA,net_ESP,net_total
143694,2005-01-01,1.0,2000.0,0.0,3093.0,-874.0,2432.0,0.0,1405.0,-22.0,NaN,NaN,-1966.0,2288.0,1.0,2005-01-01 01:00:00,2005.0,2005-01,2005-01-01,2000.0,3967.0,2432.0,1427.0,9826.0
166337,2005-01-01,2.0,2000.0,0.0,3093.0,-814.0,2432.0,0.0,1405.0,-22.0,NaN,NaN,-2045.0,2238.0,2.0,2005-01-01 02:00:00,2005.0,2005-01,2005-01-01,2000.0,3907.0,2432.0,1427.0,9766.0
9956,2005-01-01,3.0,2000.0,0.0,3093.0,-750.0,2432.0,0.0,1405.0,-22.0,NaN,NaN,-2057.0,2238.0,3.0,2005-01-01 03:00:00,2005.0,2005-01,2005-01-01,2000.0,3843.0,2432.0,1427.0,9702.0
76821,2005-01-01,4.0,2000.0,0.0,3113.0,-698.0,2432.0,0.0,1405.0,-22.0,NaN,NaN,-2038.0,2010.0,4.0,2005-01-01 04:00:00,2005.0,2005-01,2005-01-01,2000.0,3811.0,2432.0,1427.0,9670.0
126009,2005-01-01,5.0,2000.0,0.0,3113.0,-693.0,2432.0,0.0,1405.0,-22.0,NaN,NaN,-2026.0,2010.0,5.0,2005-01-01 05:00:00,2005.0,2005-01,2005-01-01,2000.0,3806.0,2432.0,1427.0,9665.0


In [7]:
monthly = df.groupby("month")["net_total"].sum()
hourly = df.groupby("hour")["net_total"].mean()
partner_totals = {p: df[f"net_{p}"].sum() for p in ["GBR","CHE","ITA","ESP"] if f"net_{p}" in df.columns}

monthly.tail(), partner_totals

(month
 2024-09    3.187217e+06
 2024-10    3.148125e+06
 2024-11    3.102896e+06
 2024-12    2.955399e+06
 NaT        9.750689e+08
 Name: net_total, dtype: float64,
 {'GBR': np.float64(333594300.73),
  'CHE': np.float64(587673663.54),
  'ITA': np.float64(398001446.04999995),
  'ESP': np.float64(287602890.22)})

In [10]:
# 🧾 Sauvegarde du dataset nettoyé et enrichi
output_path = "../data/processed/processed-imports-exports.csv"

# Créer le dossier s’il n’existe pas
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Sauvegarde au format CSV (UTF-8 + séparateur ;)
df.to_csv(output_path, sep=";", index=False, encoding="utf-8")

print(f"✅ Fichier exporté avec succès : {output_path}")
print(f"Nombre de lignes : {len(df):,}")
print(f"Colonnes : {len(df.columns)}")

✅ Fichier exporté avec succès : ../data/processed/processed-imports-exports.csv
Nombre de lignes : 175,340
Colonnes : 24
